In [4]:
from pathlib import Path
from io import StringIO
import re
import pandas as pd

in_path  = Path("ost_data_raw.csv")
out_path = in_path.with_name("ost_data.csv")

expected_cols = [
    "Data","Ora","Shkembimi skeduluar nderkufitar","Shkembimi fizik nderkufitar",
    "Frekuenca","Prodhimi Total","Ngarkesa Totale","AGC Integral of ACE",
    "Shkembimi fizik nderkufitar AL-ME","Shkembimi skeduluar nderkufitar AL-ME",
    "Shkembimi fizik nderkufitar AL-KS","Shkembimi skeduluar nderkufitar AL-KS",
    "Shkembimi fizik nderkufitar AL-GR","Shkembimi skeduluar nderkufitar AL-GR"
]

raw = in_path.read_text(encoding="utf-8", errors="ignore")
lines = raw.splitlines()

hdr = lines[0]
if hdr.count(";") < 3 and hdr.count(",") >= 3:
    hdr = hdr.replace(",", ";")

fixed_lines = [hdr]
for ln in lines[1:]:
    ln = ln.rstrip()
    if not ln:
        continue
    ln = re.sub(r",+\s*$", "", ln)   # hiq presjet fundore
    fixed_lines.append(ln)

buf = StringIO("\n".join(fixed_lines))
df = pd.read_csv(buf, sep=";", engine="python")

if len(df.columns) != len(expected_cols):
    df = df.iloc[:, :len(expected_cols)].copy()
df.columns = expected_cols

# ✔ FIX: përdor .str.strip() në vend të .strip()
df["Ora"] = df["Ora"].astype(str).str.strip().str.upper()

df.to_csv(out_path, sep=";", index=False, encoding="utf-8")
print(f"OK -> {out_path} | rows={len(df)} cols={df.shape[1]}")



OK -> ost_data.csv | rows=11743 cols=14
